<a href="https://colab.research.google.com/github/cvs2010/DeepLearningProjects/blob/main/Projeto_5_scraping_data_zapimoveis_df.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importando as bibliotecas

In [2]:
# Importando as bibliotecas
import bs4
from bs4 import BeautifulSoup
import urllib.request as urllib_request
from urllib.request import Request, urlopen, urlretrieve
from urllib.request import Request, urlopen
from urllib.error import URLError, HTTPError
import urllib.parse
import pandas as pd
import re
import requests
import math
from numpy.lib.function_base import quantile

print("BeautifulSoup ->", bs4.__version__)
print("urllib ->", urllib_request.__version__)
print("pandas ->", pd.__version__)

BeautifulSoup -> 4.11.2
urllib -> 3.10
pandas -> 1.5.3


# Webscrapping de imóveis a venda em Zapimoveis

In [ ]:
# Declarando variáveis qtd_imoveis
qtd_imoveis = []

# Iterando na página
pagestimes = int(1)

# Iterando por todas as páginas do site

# Obtendo o HTML
response = 'https://www.zapimoveis.com.br/venda/imoveis/?pagina=1'
headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.71 Safari/537.36'}

try:
    response = requests.get(response, headers=headers)
    soup = BeautifulSoup(response.content, 'html.parser')
except requests.exceptions.RequestException as e:
    print("Erro na requisição:", e)

# Obtendo a quantidade de imóveis
quantidade_imoveis_tag = soup.find('div', {'class': 'listing-wrapper__title'})
quantidade_imoveis_text = quantidade_imoveis_tag.get_text()
quantidade_imoveis_text = ''.join(filter(str.isdigit, quantidade_imoveis_text))
quantidade_imoveis = int(quantidade_imoveis_text)

# Obtendo o número de páginas
if quantidade_imoveis > 10000:
    quantidade_paginas_tag = 100
else:
    quantidade_paginas_tag = math.ceil(quantidade_imoveis / 100) - 1

#Declarando variável cards
cards = []
garagem = []
quartos = []
rua = []
bairro = []

##Obtendo HTML e o total de paginas
pages = int(quantidade_paginas_tag)


##interando por todas as paginas do site
for i in range(pages):
    ##Obtendo o HTML

    response = 'https://www.zapimoveis.com.br/venda/imoveis/?pagina='+str(i)
    headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.71 Safari/537.36'}

    requisicao = Request(response,headers = headers)

    try:
        response = urlopen(requisicao)
        html = response.read().decode('utf-8')
        soup = BeautifulSoup(html, 'html.parser')
    except HTTPError as e:
        print(e.code, e.reason)
    except URLError as e:
        print(e.reason)


    #Obtendo as Tags de interesse
    container = soup.find('div', {'class': 'listings__container'})
    if container is not None:
        anuncios = container.findAll('div', class_='card-listing simple-card')
        # Restante do código para coletar informações dos anúncios
    else:
        print("Elemento não encontrado")

    #Coletando as informações dos CARDS
    for anuncio in anuncios:
        card = {}

        # Valor
        valor = anuncio.find('p', {'class': 'simple-card__price js-price color-darker heading-regular heading-regular__bolder align-left'})
        if valor is None:
            card['valor'] = 0
        else:
            text = valor.get_text().replace('R$', '').replace('\n', '').replace('.', '').strip()
            if text == 'Sob consulta':
                card['valor'] = 0
            else:
                card['valor'] = int(text)


        #descricao
        descricao = anuncio.find('span',{'class':'simple-card__text text-regular'})
        if descricao == None:
            card['descricao'] = str('Vazio')
        else :
            card['descricao'] = str(descricao.get_text().replace('-',' ').replace('\n',''))


        #endereco
        endereco = anuncio.find('h2',{'class':'simple-card__address color-dark text-regular'})
        if endereco == None:
            card['local'], card['bairro/cidade'] = 'Não disponivel', 'Não disponivel'
        else :
            card['local'], card['bairro/cidade'], *outro = endereco.get_text().replace('\n','').replace('  ','').split(',')


        #quartos
        quartos = anuncio.find('span', {'itemprop': 'numberOfRooms'})
        if quartos is None:
            card['quartos'] = 1
        else:
            text = quartos.get_text().replace(' ', '').replace('\n', '')
            if '-' in text:
                quartos_range = text.split('-')
                card['quartos'] = [int(q) for q in quartos_range]
            else:
                card['quartos'] = int(text)



        #vagas
        garagem = anuncio.find('li', {'class': 'feature__item text-small js-parking-spaces'})
        if garagem is None:
            card['garagem'] = 0
        else:
            text = garagem.get_text().replace(' ', '').replace('\n', '')
            digits = re.findall(r'\d+', text)
            if digits:
                card['garagem'] = int(digits[0])
            else:
                card['garagem'] = 0


        #banheiros
        banheiro = anuncio.find('span',{'itemprop':'numberOfBathroomsTotal'})
        if banheiro == None:
            card['banheiro'] = int(1)
        else :
            card['banheiro'] = int(banheiro.get_text().replace(' ','').replace('\n',''))


        #metragem
        metro = anuncio.find('span', {'itemprop': 'floorSize'})
        if metro is None:
            card['area(m2)'] = 0
        else:
            text = metro.get_text().replace(' ', '').replace('\n', '').replace('m²', '')
            digits = re.findall(r'\d+', text)
            if digits:
                card['area(m2)'] = int(digits[0])
            else:
                card['area(m2)'] = 0


        #IPTU
        iptu = anuncio.find('li',{'class':'card-price__item iptu text-regular'})
        if iptu == None:
            card['iptu'] = int('0')
        else :
            card['iptu'] = int(iptu.get_text().replace('R$','').replace('\n','').replace('.','').replace('IPTU','').replace(' ',''))


        #condominio
        condominio = anuncio.find('li', {'class': 'card-price__item condominium text-regular'})
        if condominio is None:
            card['condominio'] = 0
        else:
            text = condominio.get_text().replace('condomínioR$', '').replace(' ', '').replace('.', '')
            digits = re.findall(r'\d+', text)
            if digits:
                card['condominio'] = int(digits[0])
            else:
                card['condominio'] = 0

        #tipo
        tipo = anuncio.find('small')
        if tipo == None:
            card['novidade'] = str('Nenhuma')
        else :
            card['novidade'] = str(tipo.get_text())

         #suite
        if re.search("suite|suíte|suites|suítes", card['descricao'].lower()) != None:
            card['suite'] =  int(1)
        else:
            card['suite'] =  int(0)

        #academia
        if re.search("academia|academias", card['descricao'].lower()) != None:
            card['academia'] =  int(1)
        else:
            card['academia'] =  int(0)

        #varanda
        if re.search("varanda|terraço|terraco|sacada|varandas|terraços|terracos|sacadas", card['descricao'].lower()) != None:
            card['varanda'] =  int(1)
        else:
            card['varanda'] =  int(0)

        #transporte
        if re.search("estação|estaçao|metro|onibus|ônibus", card['descricao'].lower()) != None:
            card['transporte'] =  int(1)
        else:
            card['transporte'] =  int(0)

        #salao
        if re.search("salao|salão", card['descricao'].lower()) != None:
            card['salao'] =  int(1)
        else:
            card['salao'] =  int(0)

        #planejado
        if re.search("planejado|planejados|planejada|planejadas", card['descricao'].lower()) != None:
            card['planejado'] =  int(1)
        else:
            card['planejado'] =  int(0)

        cards.append(card)

dataset = pd.DataFrame(cards)
print(dataset.isna().sum())
dataset.to_csv('./dataset-imoveis.csv', sep=';', index=False, encoding='utf-8-sig')
dataset

AttributeError: ignored

# Webscrapping de casas em Zapimoveis

In [ ]:
# Declarando variáveis qtd_imoveis
qtd_imoveis = []

# Iterando na página
pagestimes = int(1)

# Iterando por todas as páginas do site

# Obtendo o HTML
response = 'https://www.zapimoveis.com.br/venda/casas/?pagina=1'
headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.71 Safari/537.36'}

try:
    response = requests.get(response, headers=headers)
    soup = BeautifulSoup(response.content, 'html.parser')
except requests.exceptions.RequestException as e:
    print("Erro na requisição:", e)

# Obtendo a quantidade de imóveis
quantidade_imoveis_tag = soup.find('div', {'class': 'listing-wrapper__title'})
quantidade_imoveis_text = quantidade_imoveis_tag.get_text()
quantidade_imoveis_text = ''.join(filter(str.isdigit, quantidade_imoveis_text))
quantidade_imoveis = int(quantidade_imoveis_text)

# Obtendo o número de páginas
if quantidade_imoveis > 10000:
    quantidade_paginas_tag = 100
else:
    quantidade_paginas_tag = math.ceil(quantidade_imoveis / 100) - 1

#Declarando variável cards
cards = []
garagem = []
quartos = []
rua = []
bairro = []

##Obtendo HTML e o total de paginas
pages = int(quantidade_paginas_tag)


##interando por todas as paginas do site
for i in range(pages):
    ##Obtendo o HTML

    response = 'https://www.zapimoveis.com.br/venda/casas/?pagina='+str(i)
    headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.71 Safari/537.36'}

    requisicao = Request(response,headers = headers)

    try:
        response = urlopen(requisicao)
        html = response.read().decode('utf-8')
        soup = BeautifulSoup(html, 'html.parser')
    except HTTPError as e:
        print(e.code, e.reason)
    except URLError as e:
        print(e.reason)


    #Obtendo as Tags de interesse
    container = soup.find('div', {'class': 'listings__container'})
    if container is not None:
        anuncios = container.findAll('div', class_='card-listing simple-card')
        # Restante do código para coletar informações dos anúncios
    else:
        print("Elemento não encontrado")

    #Coletando as informações dos CARDS
    for anuncio in anuncios:
        card = {}

        # Valor
        valor = anuncio.find('p', {'class': 'simple-card__price js-price color-darker heading-regular heading-regular__bolder align-left'})
        if valor is None:
            card['valor'] = 0
        else:
            text = valor.get_text().replace('R$', '').replace('\n', '').replace('.', '').strip()
            if text == 'Sob consulta':
                card['valor'] = 0
            else:
                card['valor'] = int(text)


        #descricao
        descricao = anuncio.find('span',{'class':'simple-card__text text-regular'})
        if descricao == None:
            card['descricao'] = str('Vazio')
        else :
            card['descricao'] = str(descricao.get_text().replace('-',' ').replace('\n',''))


        #endereco
        endereco = anuncio.find('h2',{'class':'simple-card__address color-dark text-regular'})
        if endereco == None:
            card['local'], card['bairro/cidade'] = 'Não disponivel', 'Não disponivel'
        else :
            card['local'], card['bairro/cidade'], *outro = endereco.get_text().replace('\n','').replace('  ','').split(',')


        #quartos
        quartos = anuncio.find('span', {'itemprop': 'numberOfRooms'})
        if quartos is None:
            card['quartos'] = 1
        else:
            text = quartos.get_text().replace(' ', '').replace('\n', '')
            if '-' in text:
                quartos_range = text.split('-')
                card['quartos'] = [int(q) for q in quartos_range]
            else:
                card['quartos'] = int(text)



        #vagas
        garagem = anuncio.find('li', {'class': 'feature__item text-small js-parking-spaces'})
        if garagem is None:
            card['garagem'] = 0
        else:
            text = garagem.get_text().replace(' ', '').replace('\n', '')
            digits = re.findall(r'\d+', text)
            if digits:
                card['garagem'] = int(digits[0])
            else:
                card['garagem'] = 0


        #banheiros
        banheiro = anuncio.find('span',{'itemprop':'numberOfBathroomsTotal'})
        if banheiro == None:
            card['banheiro'] = int(1)
        else :
            card['banheiro'] = int(banheiro.get_text().replace(' ','').replace('\n',''))


        #metragem
        metro = anuncio.find('span', {'itemprop': 'floorSize'})
        if metro is None:
            card['area(m2)'] = 0
        else:
            text = metro.get_text().replace(' ', '').replace('\n', '').replace('m²', '')
            digits = re.findall(r'\d+', text)
            if digits:
                card['area(m2)'] = int(digits[0])
            else:
                card['area(m2)'] = 0


        #IPTU
        iptu = anuncio.find('li',{'class':'card-price__item iptu text-regular'})
        if iptu == None:
            card['iptu'] = int('0')
        else :
            card['iptu'] = int(iptu.get_text().replace('R$','').replace('\n','').replace('.','').replace('IPTU','').replace(' ',''))


        #condominio
        condominio = anuncio.find('li', {'class': 'card-price__item condominium text-regular'})
        if condominio is None:
            card['condominio'] = 0
        else:
            text = condominio.get_text().replace('condomínioR$', '').replace(' ', '').replace('.', '')
            digits = re.findall(r'\d+', text)
            if digits:
                card['condominio'] = int(digits[0])
            else:
                card['condominio'] = 0

        #tipo
        tipo = anuncio.find('small')
        if tipo == None:
            card['novidade'] = str('Nenhuma')
        else :
            card['novidade'] = str(tipo.get_text())

         #suite
        if re.search("suite|suíte|suites|suítes", card['descricao'].lower()) != None:
            card['suite'] =  int(1)
        else:
            card['suite'] =  int(0)

        #academia
        if re.search("academia|academias", card['descricao'].lower()) != None:
            card['academia'] =  int(1)
        else:
            card['academia'] =  int(0)

        #varanda
        if re.search("varanda|terraço|terraco|sacada|varandas|terraços|terracos|sacadas", card['descricao'].lower()) != None:
            card['varanda'] =  int(1)
        else:
            card['varanda'] =  int(0)

        #transporte
        if re.search("estação|estaçao|metro|onibus|ônibus", card['descricao'].lower()) != None:
            card['transporte'] =  int(1)
        else:
            card['transporte'] =  int(0)

        #salao
        if re.search("salao|salão", card['descricao'].lower()) != None:
            card['salao'] =  int(1)
        else:
            card['salao'] =  int(0)

        #planejado
        if re.search("planejado|planejados|planejada|planejadas", card['descricao'].lower()) != None:
            card['planejado'] =  int(1)
        else:
            card['planejado'] =  int(0)

        cards.append(card)

dataset = pd.DataFrame(cards)
print(dataset.isna().sum())
dataset.to_csv('./dataset-casas.csv', sep=';', index=False, encoding='utf-8-sig')
dataset

500 Internal Server Error
Elemento não encontrado
Elemento não encontrado
Elemento não encontrado
valor            0
descricao        0
local            0
bairro/cidade    0
quartos          0
garagem          0
banheiro         0
area(m2)         0
iptu             0
condominio       0
novidade         0
suite            0
acabamento       0
dtype: int64


,valor,descricao,local,bairro/cidade,quartos,garagem,banheiro,area(m2),iptu,condominio,novidade,suite,acabamento
0,590000,Privilégio Imóveis vende: Excelente casa duple...,Rua Paulo José Mahfud,Vargem Pequena,3,1,3,180,76333,385,Super destaque,1,0
1,5500000,"Casa a venda na Barra da Tijuca, Condomínio Al...",Rua Augusto Camossa Saldanha,Barra da Tijuca,4,6,6,520,18000,1580,Super destaque,0,0
2,0,Casa térrea muito bem localizada no Sumarezinh...,Rua Cuiabá,Sumarezinho,3,2,2,121,80,0,Super destaque,1,0
3,60397,"Oportunidade! Casa com 43,00 m² Privativos e 1...",Centro,Guaporema,2,1,1,182,0,0,Destaque,0,0
4,240000,"Ampla casa, localizada no Jardim Campo Belo, e...",Rua Irineu dos Santos,Jardim Campo Belo,2,3,1,150,300,0,Destaque,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,390000,CASA NOVA 3Q (1 SUÍTE) JARDIM BOM JESUS PE...,Rua Maria Rodrigues de Sena Rosa,Jardim Bom Jesus,3,2,2,102,0,0,Nenhuma,1,0
496,350000,"Excelente casa linear composta por 02 quartos,...",Recanto Feliz,Barra do Piraí,2,2,2,145,350,0,Nenhuma,0,0
497,889000,"Casa com 250m² localizada no bairro Santana, P...",Rua Euclydes Moura,Santana,3,2,3,250,2500,0,Destaque,1,0
498,490000,A qualidade de vida e o conforto que você busc...,Rua Carlos Alberto Souza,Ipitanga,3,2,3,150,90,0,Destaque,0,0


# Web Scraping de apartamentos do Zapimoveis

In [ ]:
# Declarando variáveis qtd_imoveis
qtd_imoveis = []

# Iterando na página
pagestimes = int(1)

# Iterando por todas as páginas do site

# Obtendo o HTML
response = 'https://www.zapimoveis.com.br/venda/apartamentos/?pagina=1'
headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.71 Safari/537.36'}

try:
    response = requests.get(response, headers=headers)
    soup = BeautifulSoup(response.content, 'html.parser')
except requests.exceptions.RequestException as e:
    print("Erro na requisição:", e)

# Obtendo a quantidade de imóveis
quantidade_imoveis_tag = soup.find('div', {'class': 'listing-wrapper__title'})
quantidade_imoveis_text = quantidade_imoveis_tag.get_text()
quantidade_imoveis_text = ''.join(filter(str.isdigit, quantidade_imoveis_text))
quantidade_imoveis = int(quantidade_imoveis_text)

# Obtendo o número de páginas
if quantidade_imoveis > 10000:
    quantidade_paginas_tag = 100
else:
    quantidade_paginas_tag = math.ceil(quantidade_imoveis / 100) - 1

#Declarando variável cards
cards = []
garagem = []
quartos = []
rua = []
bairro = []

##Obtendo HTML e o total de paginas
pages = int(quantidade_paginas_tag)


##interando por todas as paginas do site
for i in range(pages):
    ##Obtendo o HTML

    response = 'https://www.zapimoveis.com.br/venda/apartamentos/?pagina='+str(i)
    headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.71 Safari/537.36'}

    requisicao = Request(response,headers = headers)

    try:
        response = urlopen(requisicao)
        html = response.read().decode('utf-8')
        soup = BeautifulSoup(html, 'html.parser')
    except HTTPError as e:
        print(e.code, e.reason)
    except URLError as e:
        print(e.reason)


    #Obtendo as Tags de interesse
    container = soup.find('div', {'class': 'listings__container'})
    if container is not None:
        anuncios = container.findAll('div', class_='card-listing simple-card')
        # Restante do código para coletar informações dos anúncios
    else:
        print("Elemento não encontrado")

    #Coletando as informações dos CARDS
    for anuncio in anuncios:
        card = {}

        # Valor
        valor = anuncio.find('p', {'class': 'simple-card__price js-price color-darker heading-regular heading-regular__bolder align-left'})
        if valor is None:
            card['valor'] = 0
        else:
            text = valor.get_text().replace('R$', '').replace('\n', '').replace('.', '').strip()
            if text == 'Sob consulta':
                card['valor'] = 0
            else:
                card['valor'] = int(text)


        #descricao
        descricao = anuncio.find('span',{'class':'simple-card__text text-regular'})
        if descricao == None:
            card['descricao'] = str('Vazio')
        else :
            card['descricao'] = str(descricao.get_text().replace('-',' ').replace('\n',''))


        #endereco
        endereco = anuncio.find('h2',{'class':'simple-card__address color-dark text-regular'})
        if endereco == None:
            card['local'], card['bairro/cidade'] = 'Não disponivel', 'Não disponivel'
        else :
            card['local'], card['bairro/cidade'], *outro = endereco.get_text().replace('\n','').replace('  ','').split(',')


        #quartos
        quartos = anuncio.find('span', {'itemprop': 'numberOfRooms'})
        if quartos is None:
            card['quartos'] = 1
        else:
            text = quartos.get_text().replace(' ', '').replace('\n', '')
            if '-' in text:
                quartos_range = text.split('-')
                card['quartos'] = [int(q) for q in quartos_range]
            else:
                card['quartos'] = int(text)



        #vagas
        garagem = anuncio.find('li', {'class': 'feature__item text-small js-parking-spaces'})
        if garagem is None:
            card['garagem'] = 0
        else:
            text = garagem.get_text().replace(' ', '').replace('\n', '')
            digits = re.findall(r'\d+', text)
            if digits:
                card['garagem'] = int(digits[0])
            else:
                card['garagem'] = 0


        #banheiros
        banheiro = anuncio.find('span',{'itemprop':'numberOfBathroomsTotal'})
        if banheiro == None:
            card['banheiro'] = int(1)
        else :
            card['banheiro'] = int(banheiro.get_text().replace(' ','').replace('\n',''))


        #metragem
        metro = anuncio.find('span', {'itemprop': 'floorSize'})
        if metro is None:
            card['area(m2)'] = 0
        else:
            text = metro.get_text().replace(' ', '').replace('\n', '').replace('m²', '')
            digits = re.findall(r'\d+', text)
            if digits:
                card['area(m2)'] = int(digits[0])
            else:
                card['area(m2)'] = 0


        #IPTU
        iptu = anuncio.find('li',{'class':'card-price__item iptu text-regular'})
        if iptu == None:
            card['iptu'] = int('0')
        else :
            card['iptu'] = int(iptu.get_text().replace('R$','').replace('\n','').replace('.','').replace('IPTU','').replace(' ',''))


        #condominio
        condominio = anuncio.find('li', {'class': 'card-price__item condominium text-regular'})
        if condominio is None:
            card['condominio'] = 0
        else:
            text = condominio.get_text().replace('condomínioR$', '').replace(' ', '').replace('.', '')
            digits = re.findall(r'\d+', text)
            if digits:
                card['condominio'] = int(digits[0])
            else:
                card['condominio'] = 0

        #tipo
        tipo = anuncio.find('small')
        if tipo == None:
            card['novidade'] = str('Nenhuma')
        else :
            card['novidade'] = str(tipo.get_text())

         #suite
        if re.search("suite|suíte|suites|suítes", card['descricao'].lower()) != None:
            card['suite'] =  int(1)
        else:
            card['suite'] =  int(0)

        #academia
        if re.search("academia|academias", card['descricao'].lower()) != None:
            card['academia'] =  int(1)
        else:
            card['academia'] =  int(0)

        #varanda
        if re.search("varanda|terraço|terraco|sacada|varandas|terraços|terracos|sacadas", card['descricao'].lower()) != None:
            card['varanda'] =  int(1)
        else:
            card['varanda'] =  int(0)

        #transporte
        if re.search("estação|estaçao|metro|onibus|ônibus", card['descricao'].lower()) != None:
            card['transporte'] =  int(1)
        else:
            card['transporte'] =  int(0)

        #salao
        if re.search("salao|salão", card['descricao'].lower()) != None:
            card['salao'] =  int(1)
        else:
            card['salao'] =  int(0)

        #planejado
        if re.search("planejado|planejados|planejada|planejadas", card['descricao'].lower()) != None:
            card['planejado'] =  int(1)
        else:
            card['planejado'] =  int(0)

        cards.append(card)

dataset = pd.DataFrame(cards)
print(dataset.isna().sum())
dataset.to_csv('./dataset-ap.csv', sep=';', index=False, encoding='utf-8-sig')
dataset

500 Internal Server Error


AttributeError: ignored

# Web Scraping de imóveis no DF do Zapimoveis

In [ ]:
# Declarando variáveis qtd_imoveis
qtd_imoveis = []

# Iterando na página
pagestimes = int(1)

# Iterando por todas as páginas do site

# Obtendo o HTML
response = 'https://www.zapimoveis.com.br/venda/imoveis/df+brasilia/?pagina=1'
headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.71 Safari/537.36'}

try:
    response = requests.get(response, headers=headers)
    soup = BeautifulSoup(response.content, 'html.parser')
except requests.exceptions.RequestException as e:
    print("Erro na requisição:", e)

# Obtendo a quantidade de imóveis
quantidade_imoveis_tag = soup.find('div', {'class': 'listing-wrapper__title'})
quantidade_imoveis_text = quantidade_imoveis_tag.get_text()
quantidade_imoveis_text = ''.join(filter(str.isdigit, quantidade_imoveis_text))
quantidade_imoveis = int(quantidade_imoveis_text)

# Obtendo o número de páginas
if quantidade_imoveis > 10000:
    quantidade_paginas_tag = 100
else:
    quantidade_paginas_tag = math.ceil(quantidade_imoveis / 100) - 1

#Declarando variável cards
cards = []
garagem = []
quartos = []
rua = []
bairro = []

##Obtendo HTML e o total de paginas
pages = int(quantidade_paginas_tag)


##interando por todas as paginas do site
for i in range(pages):
    ##Obtendo o HTML

    response = 'https://www.zapimoveis.com.br/venda/imoveis/df+brasilia/?pagina='+str(i)
    headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.71 Safari/537.36'}

    requisicao = Request(response,headers = headers)

    try:
        response = urlopen(requisicao)
        html = response.read().decode('utf-8')
        soup = BeautifulSoup(html, 'html.parser')
    except HTTPError as e:
        print(e.code, e.reason)
    except URLError as e:
        print(e.reason)


    #Obtendo as Tags de interesse
    container = soup.find('div', {'class': 'listings__container'})
    if container is not None:
        anuncios = container.findAll('div', class_='card-listing simple-card')
        # Restante do código para coletar informações dos anúncios
    else:
        print("Elemento não encontrado")

    #Coletando as informações dos CARDS
    for anuncio in anuncios:
        card = {}

        # Valor
        valor = anuncio.find('p', {'class': 'simple-card__price js-price color-darker heading-regular heading-regular__bolder align-left'})
        if valor is None:
            card['valor'] = 0
        else:
            text = valor.get_text().replace('R$', '').replace('\n', '').replace('.', '').strip()
            if text == 'Sob consulta':
                card['valor'] = 0
            else:
                card['valor'] = int(text)


        #descricao
        descricao = anuncio.find('span',{'class':'simple-card__text text-regular'})
        if descricao == None:
            card['descricao'] = str('Vazio')
        else :
            card['descricao'] = str(descricao.get_text().replace('-',' ').replace('\n',''))


        #endereco
        endereco = anuncio.find('h2',{'class':'simple-card__address color-dark text-regular'})
        if endereco == None:
            card['local'], card['bairro/cidade'] = 'Não disponivel', 'Não disponivel'
        else :
            card['local'], card['bairro/cidade'], *outro = endereco.get_text().replace('\n','').replace('  ','').split(',')


        #quartos
        quartos = anuncio.find('span', {'itemprop': 'numberOfRooms'})
        if quartos is None:
            card['quartos'] = 1
        else:
            text = quartos.get_text().replace(' ', '').replace('\n', '')
            if '-' in text:
                quartos_range = text.split('-')
                card['quartos'] = [int(q) for q in quartos_range]
            else:
                card['quartos'] = int(text)



        #vagas
        garagem = anuncio.find('li', {'class': 'feature__item text-small js-parking-spaces'})
        if garagem is None:
            card['garagem'] = 0
        else:
            text = garagem.get_text().replace(' ', '').replace('\n', '')
            digits = re.findall(r'\d+', text)
            if digits:
                card['garagem'] = int(digits[0])
            else:
                card['garagem'] = 0


        #banheiros
        banheiro = anuncio.find('span',{'itemprop':'numberOfBathroomsTotal'})
        if banheiro == None:
            card['banheiro'] = int(1)
        else :
            card['banheiro'] = int(banheiro.get_text().replace(' ','').replace('\n',''))


        #metragem
        metro = anuncio.find('span', {'itemprop': 'floorSize'})
        if metro is None:
            card['area(m2)'] = 0
        else:
            text = metro.get_text().replace(' ', '').replace('\n', '').replace('m²', '')
            digits = re.findall(r'\d+', text)
            if digits:
                card['area(m2)'] = int(digits[0])
            else:
                card['area(m2)'] = 0


        #IPTU
        iptu = anuncio.find('li',{'class':'card-price__item iptu text-regular'})
        if iptu == None:
            card['iptu'] = int('0')
        else :
            card['iptu'] = int(iptu.get_text().replace('R$','').replace('\n','').replace('.','').replace('IPTU','').replace(' ',''))


        #condominio
        condominio = anuncio.find('li', {'class': 'card-price__item condominium text-regular'})
        if condominio is None:
            card['condominio'] = 0
        else:
            text = condominio.get_text().replace('condomínioR$', '').replace(' ', '').replace('.', '')
            digits = re.findall(r'\d+', text)
            if digits:
                card['condominio'] = int(digits[0])
            else:
                card['condominio'] = 0

        #tipo
        tipo = anuncio.find('small')
        if tipo == None:
            card['novidade'] = str('Nenhuma')
        else :
            card['novidade'] = str(tipo.get_text())

         #suite
        if re.search("suite|suíte|suites|suítes", card['descricao'].lower()) != None:
            card['suite'] =  int(1)
        else:
            card['suite'] =  int(0)

        #academia
        if re.search("academia|academias", card['descricao'].lower()) != None:
            card['academia'] =  int(1)
        else:
            card['academia'] =  int(0)

        #varanda
        if re.search("varanda|terraço|terraco|sacada|varandas|terraços|terracos|sacadas", card['descricao'].lower()) != None:
            card['varanda'] =  int(1)
        else:
            card['varanda'] =  int(0)

        #transporte
        if re.search("estação|estaçao|metro|onibus|ônibus", card['descricao'].lower()) != None:
            card['transporte'] =  int(1)
        else:
            card['transporte'] =  int(0)

        #salao
        if re.search("salao|salão", card['descricao'].lower()) != None:
            card['salao'] =  int(1)
        else:
            card['salao'] =  int(0)

        #planejado
        if re.search("planejado|planejados|planejada|planejadas", card['descricao'].lower()) != None:
            card['planejado'] =  int(1)
        else:
            card['planejado'] =  int(0)

        cards.append(card)

dataset = pd.DataFrame(cards)
print(dataset.isna().sum())
dataset.to_csv('./dataset-imovel-df.csv', sep=';', index=False, encoding='utf-8-sig')
dataset

500 Internal Server Error
Elemento não encontrado
Elemento não encontrado
Elemento não encontrado
Elemento não encontrado
Elemento não encontrado
Elemento não encontrado
Elemento não encontrado
Elemento não encontrado
Elemento não encontrado
Elemento não encontrado
Elemento não encontrado
Elemento não encontrado
Elemento não encontrado
Elemento não encontrado
Elemento não encontrado
Elemento não encontrado
Elemento não encontrado
Elemento não encontrado
Elemento não encontrado
Elemento não encontrado
Elemento não encontrado
Elemento não encontrado
Elemento não encontrado
Elemento não encontrado
Elemento não encontrado
Elemento não encontrado
Elemento não encontrado
Elemento não encontrado
Elemento não encontrado
Elemento não encontrado
Elemento não encontrado
Elemento não encontrado
Elemento não encontrado
Elemento não encontrado
Elemento não encontrado
Elemento não encontrado
Elemento não encontrado
Elemento não encontrado
Elemento não encontrado
Elemento não encontrado
Elemento não e

,valor,descricao,local,bairro/cidade,quartos,garagem,banheiro,area(m2),iptu,condominio,novidade,suite,academia,varanda,transporte,salao,planejado
0,1150000,Prédio com 450m² para investidores! Com renda ...,Riacho Fundo I,Brasília,1,0,4,450,0,0,Super destaque,0,0,0,0,0,0
1,3650000,Top Line Prime Imóveis exclusivos para cliente...,Avenida Parque Águas Claras,Norte,4,4,6,285,0,2400,Super destaque,1,0,1,0,0,0
2,160000,"LOTE COM UMA PEQUENA COZINHA SIMPLES, GRADE E ...",Quadra 303 Conjunto 6,Recanto das Emas,1,4,1,150,0,0,Super destaque,0,0,0,0,0,0
3,870000,COELHO IMÓVEIS CONSULTORIA PRIME VENDE: Belis...,CA 5,Lago Norte,3,1,2,112,1435,959,Nenhuma,0,0,0,0,0,0
4,550000,Onde nos encontrar? Instagram: @oalmeidaimovei...,Ceilandia Norte,Brasília,3,4,1,250,0,0,Nenhuma,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,380000,Vendo ótima casa em caldas novas de GO!! Casa ...,SQS 216 Bloco J,Asa Sul,3,4,2,410,0,0,Nenhuma,1,0,0,1,0,1
19996,490000,A casa no bairro Residencial Santos Dumont pos...,QRI 27,Residencial Santos Dumont,3,0,2,150,0,0,Nenhuma,1,0,1,1,0,0
19997,320000,Onde nos encontrar? Instagram: @oalmeidaimovei...,Ceilandia Norte,Brasília,3,1,2,70,0,330,Nenhuma,0,0,0,0,0,0
19998,250000,CNB 13 Apartamento2 Quartos Vaga Nascente À ...,CNB 13,Taguatinga Norte,2,1,1,65,0,420,Nenhuma,0,0,0,0,0,0


# Web Scraping de casas no DF do Zapimoveis

In [ ]:
# Declarando variáveis qtd_imoveis
qtd_imoveis = []

# Iterando na página
pagestimes = int(1)

# Iterando por todas as páginas do site

# Obtendo o HTML
response = 'https://www.zapimoveis.com.br/venda/casas/df+brasilia/?pagina=1'
headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.71 Safari/537.36'}

try:
    response = requests.get(response, headers=headers)
    soup = BeautifulSoup(response.content, 'html.parser')
except requests.exceptions.RequestException as e:
    print("Erro na requisição:", e)

# Obtendo a quantidade de imóveis
quantidade_imoveis_tag = soup.find('div', {'class': 'listing-wrapper__title'})
quantidade_imoveis_text = quantidade_imoveis_tag.get_text()
quantidade_imoveis_text = ''.join(filter(str.isdigit, quantidade_imoveis_text))
quantidade_imoveis = int(quantidade_imoveis_text)

# Obtendo o número de páginas
if quantidade_imoveis > 10000:
    quantidade_paginas_tag = 100
else:
    quantidade_paginas_tag = math.ceil(quantidade_imoveis / 100) - 1

#Declarando variável cards
cards = []
garagem = []
quartos = []
rua = []
bairro = []

##Obtendo HTML e o total de paginas
pages = int(quantidade_paginas_tag)


##interando por todas as paginas do site
for i in range(pages):
    ##Obtendo o HTML

    response = 'https://www.zapimoveis.com.br/venda/casas/df+brasilia/?pagina='+str(i)
    headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.71 Safari/537.36'}

    requisicao = Request(response,headers = headers)

    try:
        response = urlopen(requisicao)
        html = response.read().decode('utf-8')
        soup = BeautifulSoup(html, 'html.parser')
    except HTTPError as e:
        print(e.code, e.reason)
    except URLError as e:
        print(e.reason)


    #Obtendo as Tags de interesse
    container = soup.find('div', {'class': 'listings__container'})
    if container is not None:
        anuncios = container.findAll('div', class_='card-listing simple-card')
        # Restante do código para coletar informações dos anúncios
    else:
        print("Elemento não encontrado")

    #Coletando as informações dos CARDS
    for anuncio in anuncios:
        card = {}

        # Valor
        valor = anuncio.find('p', {'class': 'simple-card__price js-price color-darker heading-regular heading-regular__bolder align-left'})
        if valor is None:
            card['valor'] = 0
        else:
            text = valor.get_text().replace('R$', '').replace('\n', '').replace('.', '').strip()
            if text == 'Sob consulta':
                card['valor'] = 0
            else:
                card['valor'] = int(text)


        #descricao
        descricao = anuncio.find('span',{'class':'simple-card__text text-regular'})
        if descricao == None:
            card['descricao'] = str('Vazio')
        else :
            card['descricao'] = str(descricao.get_text().replace('-',' ').replace('\n',''))


        #endereco
        endereco = anuncio.find('h2',{'class':'simple-card__address color-dark text-regular'})
        if endereco == None:
            card['local'], card['bairro/cidade'] = 'Não disponivel', 'Não disponivel'
        else :
            card['local'], card['bairro/cidade'], *outro = endereco.get_text().replace('\n','').replace('  ','').split(',')


        #quartos
        quartos = anuncio.find('span', {'itemprop': 'numberOfRooms'})
        if quartos is None:
            card['quartos'] = 1
        else:
            text = quartos.get_text().replace(' ', '').replace('\n', '')
            if '-' in text:
                quartos_range = text.split('-')
                card['quartos'] = [int(q) for q in quartos_range]
            else:
                card['quartos'] = int(text)



        #vagas
        garagem = anuncio.find('li', {'class': 'feature__item text-small js-parking-spaces'})
        if garagem is None:
            card['garagem'] = 0
        else:
            text = garagem.get_text().replace(' ', '').replace('\n', '')
            digits = re.findall(r'\d+', text)
            if digits:
                card['garagem'] = int(digits[0])
            else:
                card['garagem'] = 0


        #banheiros
        banheiro = anuncio.find('span',{'itemprop':'numberOfBathroomsTotal'})
        if banheiro == None:
            card['banheiro'] = int(1)
        else :
            card['banheiro'] = int(banheiro.get_text().replace(' ','').replace('\n',''))


        #metragem
        metro = anuncio.find('span', {'itemprop': 'floorSize'})
        if metro is None:
            card['area(m2)'] = 0
        else:
            text = metro.get_text().replace(' ', '').replace('\n', '').replace('m²', '')
            digits = re.findall(r'\d+', text)
            if digits:
                card['area(m2)'] = int(digits[0])
            else:
                card['area(m2)'] = 0


        #IPTU
        iptu = anuncio.find('li',{'class':'card-price__item iptu text-regular'})
        if iptu == None:
            card['iptu'] = int('0')
        else :
            card['iptu'] = int(iptu.get_text().replace('R$','').replace('\n','').replace('.','').replace('IPTU','').replace(' ',''))


        #condominio
        condominio = anuncio.find('li', {'class': 'card-price__item condominium text-regular'})
        if condominio is None:
            card['condominio'] = 0
        else:
            text = condominio.get_text().replace('condomínioR$', '').replace(' ', '').replace('.', '')
            digits = re.findall(r'\d+', text)
            if digits:
                card['condominio'] = int(digits[0])
            else:
                card['condominio'] = 0

        #tipo
        tipo = anuncio.find('small')
        if tipo == None:
            card['novidade'] = str('Nenhuma')
        else :
            card['novidade'] = str(tipo.get_text())

         #suite
        if re.search("suite|suíte|suites|suítes", card['descricao'].lower()) != None:
            card['suite'] =  int(1)
        else:
            card['suite'] =  int(0)

        #academia
        if re.search("academia|academias", card['descricao'].lower()) != None:
            card['academia'] =  int(1)
        else:
            card['academia'] =  int(0)

        #varanda
        if re.search("varanda|terraço|terraco|sacada|varandas|terraços|terracos|sacadas", card['descricao'].lower()) != None:
            card['varanda'] =  int(1)
        else:
            card['varanda'] =  int(0)

        #transporte
        if re.search("estação|estaçao|metro|onibus|ônibus", card['descricao'].lower()) != None:
            card['transporte'] =  int(1)
        else:
            card['transporte'] =  int(0)

        #salao
        if re.search("salao|salão", card['descricao'].lower()) != None:
            card['salao'] =  int(1)
        else:
            card['salao'] =  int(0)

        #planejado
        if re.search("planejado|planejados|planejada|planejadas", card['descricao'].lower()) != None:
            card['planejado'] =  int(1)
        else:
            card['planejado'] =  int(0)

        cards.append(card)

dataset = pd.DataFrame(cards)
print(dataset.isna().sum())
dataset.to_csv('./dataset-casas-df.csv', sep=';', index=False, encoding='utf-8-sig')
dataset

500 Internal Server Error
Elemento não encontrado
Elemento não encontrado
Elemento não encontrado
Elemento não encontrado
Elemento não encontrado
Elemento não encontrado
Elemento não encontrado
Elemento não encontrado
Elemento não encontrado
Elemento não encontrado
Elemento não encontrado
Elemento não encontrado
Elemento não encontrado
Elemento não encontrado
Elemento não encontrado
Elemento não encontrado
Elemento não encontrado
Elemento não encontrado
Elemento não encontrado
Elemento não encontrado
Elemento não encontrado
Elemento não encontrado
Elemento não encontrado
Elemento não encontrado
Elemento não encontrado
Elemento não encontrado
Elemento não encontrado
Elemento não encontrado
Elemento não encontrado
Elemento não encontrado
Elemento não encontrado
Elemento não encontrado
Elemento não encontrado
Elemento não encontrado
Elemento não encontrado
Elemento não encontrado
Elemento não encontrado
Elemento não encontrado
Elemento não encontrado
Elemento não encontrado
Elemento não e

,valor,descricao,local,bairro/cidade,quartos,garagem,banheiro,area(m2),iptu,condominio,novidade,suite,academia,varanda,transporte,salao,planejado
0,5999,O lançamento mais esperado!! * CONDOMÍNIO BOR...,SMDB Conjunto 7,Setor de Mansoes Dom Bosco,1,0,1,1000,0,0,Nenhuma,0,0,0,0,0,0
1,279000,"ÓTIMA CASA A VENDA COM 2 QTS, SALA, COZINHA, 2...",SHA Conjunto 5 Chácara 116,Setor Habitacional Arniqueira,2,4,2,300,500,150,Nenhuma,0,0,1,0,0,0
2,327000,DESCRIÇÃO DO LOTE: A casa da frente possui doi...,QNM 5 Conjunto F,Ceilandia Sul,2,2,2,180,0,0,Nenhuma,0,0,0,0,0,0
3,315000,Excelente oportunidade!! Vendo um dos melhores...,Quadra 3,Jardim Roriz,4,2,2,128,0,0,Nenhuma,0,0,0,0,0,0
4,1100000,O lote/terreno no bairro Setor Habitacional Vi...,Rua 10 Chácara 154/1,Setor Habitacional Vicente Pires,1,0,1,1015,0,0,Nenhuma,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15042,3950000,DALL'OCA IMÓVEIS VENDE: Ótima casa em Ponta S...,SHIS QI 25 Conjunto 13,Lago Sul,5,4,5,565,9807,0,Nenhuma,0,0,0,0,0,0
15043,2500000,"Com suntuosidade e elegância, esta casa térrea...",Setor de Habitações Individuais Norte,Brasília,4,6,2,360,0,0,Nenhuma,0,0,0,0,0,0
15044,700000,Edícula Condomínio Estancia Quintas da Alvor...,Condomínio Estância Quintas da Alvorada,Setor Habitacional Jardim Botânico,1,0,1,50,1500,405,Nenhuma,0,0,0,0,0,0
15045,770000,Casa Localizada no Condomínio Estancia Quintas...,Condomínio Estância Quintas da Alvorada,Setor Habitacional Jardim Botânico,1,1,1,40,1500,405,Nenhuma,0,0,0,0,0,0


# Web Scraping de apartamentos no DF do Zapimoveis

In [ ]:
# Declarando variáveis qtd_imoveis
qtd_imoveis = []

# Iterando na página
pagestimes = int(1)

# Iterando por todas as páginas do site

# Obtendo o HTML
response = 'https://www.zapimoveis.com.br/venda/apartamentos/df+brasilia/?pagina=1'
headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.71 Safari/537.36'}

try:
    response = requests.get(response, headers=headers)
    soup = BeautifulSoup(response.content, 'html.parser')
except requests.exceptions.RequestException as e:
    print("Erro na requisição:", e)

# Obtendo a quantidade de imóveis
quantidade_imoveis_tag = soup.find('div', {'class': 'listing-wrapper__title'})
quantidade_imoveis_text = quantidade_imoveis_tag.get_text()
quantidade_imoveis_text = ''.join(filter(str.isdigit, quantidade_imoveis_text))
quantidade_imoveis = int(quantidade_imoveis_text)

# Obtendo o número de páginas
if quantidade_imoveis > 10000:
    quantidade_paginas_tag = 100
else:
    quantidade_paginas_tag = math.ceil(quantidade_imoveis / 100) - 1

#Declarando variável cards
cards = []
garagem = []
quartos = []
rua = []
bairro = []

##Obtendo HTML e o total de paginas
pages = int(quantidade_paginas_tag)


##interando por todas as paginas do site
for i in range(pages):
    ##Obtendo o HTML

    response = 'https://www.zapimoveis.com.br/venda/apartamentos/df+brasilia/?pagina='+str(i)
    headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.71 Safari/537.36'}

    requisicao = Request(response,headers = headers)

    try:
        response = urlopen(requisicao)
        html = response.read().decode('utf-8')
        soup = BeautifulSoup(html, 'html.parser')
    except HTTPError as e:
        print(e.code, e.reason)
    except URLError as e:
        print(e.reason)


    #Obtendo as Tags de interesse
    container = soup.find('div', {'class': 'listings__container'})
    if container is not None:
        anuncios = container.findAll('div', class_='card-listing simple-card')
        # Restante do código para coletar informações dos anúncios
    else:
        print("Elemento não encontrado")

    #Coletando as informações dos CARDS
    for anuncio in anuncios:
        card = {}

        # Valor
        valor = anuncio.find('p', {'class': 'simple-card__price js-price color-darker heading-regular heading-regular__bolder align-left'})
        if valor is None:
            card['valor'] = 0
        else:
            text = valor.get_text().replace('R$', '').replace('\n', '').replace('.', '').strip()
            if text == 'Sob consulta':
                card['valor'] = 0
            else:
                card['valor'] = int(text)


        #descricao
        descricao = anuncio.find('span',{'class':'simple-card__text text-regular'})
        if descricao == None:
            card['descricao'] = str('Vazio')
        else :
            card['descricao'] = str(descricao.get_text().replace('-',' ').replace('\n',''))


        #endereco
        endereco = anuncio.find('h2',{'class':'simple-card__address color-dark text-regular'})
        if endereco == None:
            card['local'], card['bairro/cidade'] = 'Não disponivel', 'Não disponivel'
        else :
            card['local'], card['bairro/cidade'], *outro = endereco.get_text().replace('\n','').replace('  ','').split(',')


        #quartos
        quartos = anuncio.find('span', {'itemprop': 'numberOfRooms'})
        if quartos is None:
            card['quartos'] = 1
        else:
            text = quartos.get_text().replace(' ', '').replace('\n', '')
            if '-' in text:
                quartos_range = text.split('-')
                card['quartos'] = [int(q) for q in quartos_range]
            else:
                card['quartos'] = int(text)



        #vagas
        garagem = anuncio.find('li', {'class': 'feature__item text-small js-parking-spaces'})
        if garagem is None:
            card['garagem'] = 0
        else:
            text = garagem.get_text().replace(' ', '').replace('\n', '')
            digits = re.findall(r'\d+', text)
            if digits:
                card['garagem'] = int(digits[0])
            else:
                card['garagem'] = 0


        #banheiros
        banheiro = anuncio.find('span',{'itemprop':'numberOfBathroomsTotal'})
        if banheiro == None:
            card['banheiro'] = int(1)
        else :
            card['banheiro'] = int(banheiro.get_text().replace(' ','').replace('\n',''))


        #metragem
        metro = anuncio.find('span', {'itemprop': 'floorSize'})
        if metro is None:
            card['area(m2)'] = 0
        else:
            text = metro.get_text().replace(' ', '').replace('\n', '').replace('m²', '')
            digits = re.findall(r'\d+', text)
            if digits:
                card['area(m2)'] = int(digits[0])
            else:
                card['area(m2)'] = 0


        #IPTU
        iptu = anuncio.find('li',{'class':'card-price__item iptu text-regular'})
        if iptu == None:
            card['iptu'] = int('0')
        else :
            card['iptu'] = int(iptu.get_text().replace('R$','').replace('\n','').replace('.','').replace('IPTU','').replace(' ',''))


        #condominio
        condominio = anuncio.find('li', {'class': 'card-price__item condominium text-regular'})
        if condominio is None:
            card['condominio'] = 0
        else:
            text = condominio.get_text().replace('condomínioR$', '').replace(' ', '').replace('.', '')
            digits = re.findall(r'\d+', text)
            if digits:
                card['condominio'] = int(digits[0])
            else:
                card['condominio'] = 0

        #tipo
        tipo = anuncio.find('small')
        if tipo == None:
            card['novidade'] = str('Nenhuma')
        else :
            card['novidade'] = str(tipo.get_text())

         #suite
        if re.search("suite|suíte|suites|suítes", card['descricao'].lower()) != None:
            card['suite'] =  int(1)
        else:
            card['suite'] =  int(0)

        #academia
        if re.search("academia|academias", card['descricao'].lower()) != None:
            card['academia'] =  int(1)
        else:
            card['academia'] =  int(0)

        #varanda
        if re.search("varanda|terraço|terraco|sacada|varandas|terraços|terracos|sacadas", card['descricao'].lower()) != None:
            card['varanda'] =  int(1)
        else:
            card['varanda'] =  int(0)

        #transporte
        if re.search("estação|estaçao|metro|onibus|ônibus", card['descricao'].lower()) != None:
            card['transporte'] =  int(1)
        else:
            card['transporte'] =  int(0)

        #salao
        if re.search("salao|salão", card['descricao'].lower()) != None:
            card['salao'] =  int(1)
        else:
            card['salao'] =  int(0)

        #planejado
        if re.search("planejado|planejados|planejada|planejadas", card['descricao'].lower()) != None:
            card['planejado'] =  int(1)
        else:
            card['planejado'] =  int(0)

        cards.append(card)

dataset = pd.DataFrame(cards)
print(dataset.isna().sum())
dataset.to_csv('./dataset-ap-df.csv', sep=';', index=False, encoding='utf-8-sig')
dataset

Elemento não encontrado
Elemento não encontrado
Elemento não encontrado
Elemento não encontrado
Elemento não encontrado
Elemento não encontrado
Elemento não encontrado
Elemento não encontrado
Elemento não encontrado
Elemento não encontrado
Elemento não encontrado
Elemento não encontrado
Elemento não encontrado
Elemento não encontrado
Elemento não encontrado
Elemento não encontrado
Elemento não encontrado
Elemento não encontrado
Elemento não encontrado
Elemento não encontrado
Elemento não encontrado
Elemento não encontrado
Elemento não encontrado
Elemento não encontrado
Elemento não encontrado
Elemento não encontrado
Elemento não encontrado
Elemento não encontrado
Elemento não encontrado
Elemento não encontrado
Elemento não encontrado
Elemento não encontrado
Elemento não encontrado
Elemento não encontrado
Elemento não encontrado
Elemento não encontrado
valor            0
descricao        0
local            0
bairro/cidade    0
quartos          0
garagem          0
banheiro         0
are

,valor,descricao,local,bairro/cidade,quartos,garagem,banheiro,area(m2),iptu,condominio,novidade,suite,academia,varanda,transporte,salao,planejado
0,606900,"Viva próximo de tudo o que você deseja, na mel...",Rua 3,Norte,2,1,2,65,0,0,Na planta,0,0,0,1,0,0
1,1400000,"Viva próximo de tudo o que você deseja, na mel...",Rua 3,Norte,3,1,2,83,0,0,Na planta,0,0,0,1,0,0
2,0,O Reserva Parque Clube é assim: a combinação p...,EQS 414/415,Asa Sul,"[2, 3]",0,1,67,0,0,Em construção,0,0,0,0,0,0
3,170000,"Apartamento de um quarto, banheiro social, sal...",Guara II,Brasília,1,0,1,50,0,0,Super destaque,0,0,0,0,0,0
4,155000,ALEX GAMA VENDE: QUADRA INDUSTRIAL LESTE QI ...,Quadra 4,Setor Industrial,1,1,1,33,0,120,Super destaque,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5595,2700000,ID DESIGN 4 SUITES 4 VAGAS LAZER COMPLETO ACEI...,SQNW 303 Bloco K,Setor Noroeste,4,4,6,203,0,0,Nenhuma,1,0,0,0,0,0
5596,850000,OLYMPIQUE . 62m² . MOBILIADO . 02 QTS . VG ...,Guara II,Brasília,2,1,2,62,0,560,Nenhuma,1,0,1,0,0,0
5597,2890000,Quadra SQSW 500 Sudoeste Valor do Ágio Ap...,Setor Sudoeste,Brasília,4,4,7,393,0,0,Nenhuma,1,0,0,0,0,0
5598,1350000,Excelente cobertura na Rua 4 de Vicente Pires ...,Setor Habitacional Vicente Pires,Brasília,3,0,4,180,0,0,Nenhuma,1,0,0,0,0,0


# Testes